In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
import keras
from keras.datasets import mnist

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
LATENT_DIMS = [64]
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 3
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 512
ORIGINAL_DIM = 784
results_path = 'results\\mnist_vanillavae_dbal'
epsilon_std = 1.0

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "var_ratios": var_ratios,
    "mean_std": mean_std,
    "max_entropy": max_entropy,
    "bald": bald,
    "uniform": uniform,
#     "batch_bald": batch_bald
}


In [4]:
class MLP_REG(nn.Module):
    def __init__(self, latent_dim):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(X_train.shape[0], -1)

X_test = (X_test.astype(np.float32) - 127.5) / 127.5
X_test = X_test.reshape(X_test.shape[0], -1)

In [6]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [7]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=98,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [8]:
for latent_dim in LATENT_DIMS:
    X_train_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\mnist\x_train_enhanced_vanillavae.npy")
    X_test_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\mnist\x_test_enhanced_vanillavae.npy")
    
    for exp_iter in range(EXPERIMENT_COUNT):
        np.random.seed(exp_iter)
        initial_idx = np.array([],dtype=int)
        for i in range(10):
            idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
            initial_idx = np.concatenate((initial_idx, idx))

        for func_name, acquisition_func in ACQ_FUNCS.items():  

            X_initial = X_train_enhanced[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train_enhanced, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(latent_dim).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test_enhanced,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join(results_path, "{func_name}_latent_dim_{latent_dim}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter, latent_dim=latent_dim))
            np.save(file_name, (acc_arr, dataset_size_arr))
        '''
        for func_name, acquisition_func in ACQ_FUNCS.items():  
            X_initial = X_train[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(ORIGINAL_DIM).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join("results\\mnist_dbal", "{func_name}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter))
            np.save(file_name, (acc_arr, dataset_size_arr))
            '''

Accuracy after query 1: 0.5925
Accuracy after query 2: 0.6444
Accuracy after query 3: 0.7273
Accuracy after query 4: 0.7268
Accuracy after query 5: 0.7442
Accuracy after query 6: 0.7753
Accuracy after query 7: 0.7857
Accuracy after query 8: 0.8173
Accuracy after query 9: 0.8372
Accuracy after query 10: 0.8392
Accuracy after query 11: 0.8646
Accuracy after query 12: 0.8763
Accuracy after query 13: 0.8708
Accuracy after query 14: 0.8926
Accuracy after query 15: 0.8988
Accuracy after query 16: 0.8842
Accuracy after query 17: 0.8979
Accuracy after query 18: 0.8948
Accuracy after query 19: 0.9075
Accuracy after query 20: 0.9062
Accuracy after query 21: 0.9106
Accuracy after query 22: 0.9095
Accuracy after query 23: 0.9150
Accuracy after query 24: 0.9131
Accuracy after query 25: 0.9123
Accuracy after query 26: 0.9097
Accuracy after query 27: 0.9214
Accuracy after query 28: 0.9235
Accuracy after query 29: 0.9268
Accuracy after query 30: 0.9292
Accuracy after query 31: 0.9203
Accuracy after qu

Accuracy after query 23: 0.9142
Accuracy after query 24: 0.9270
Accuracy after query 25: 0.9288
Accuracy after query 26: 0.9329
Accuracy after query 27: 0.9380
Accuracy after query 28: 0.9352
Accuracy after query 29: 0.9411
Accuracy after query 30: 0.9341
Accuracy after query 31: 0.9356
Accuracy after query 32: 0.9333
Accuracy after query 33: 0.9394
Accuracy after query 34: 0.9347
Accuracy after query 35: 0.9335
Accuracy after query 36: 0.9339
Accuracy after query 37: 0.9326
Accuracy after query 38: 0.9430
Accuracy after query 39: 0.9420
Accuracy after query 40: 0.9415
Accuracy after query 41: 0.9467
Accuracy after query 42: 0.9427
Accuracy after query 43: 0.9498
Accuracy after query 44: 0.9480
Accuracy after query 45: 0.9477
Accuracy after query 46: 0.9432
Accuracy after query 47: 0.9438
Accuracy after query 48: 0.9461
Accuracy after query 49: 0.9467
Accuracy after query 50: 0.9460
Accuracy after query 51: 0.9485
Accuracy after query 52: 0.9475
Accuracy after query 53: 0.9533
Accuracy

Accuracy after query 45: 0.9419
Accuracy after query 46: 0.9438
Accuracy after query 47: 0.9462
Accuracy after query 48: 0.9472
Accuracy after query 49: 0.9463
Accuracy after query 50: 0.9486
Accuracy after query 51: 0.9465
Accuracy after query 52: 0.9501
Accuracy after query 53: 0.9458
Accuracy after query 54: 0.9512
Accuracy after query 55: 0.9481
Accuracy after query 56: 0.9494
Accuracy after query 57: 0.9522
Accuracy after query 58: 0.9538
Accuracy after query 59: 0.9540
Accuracy after query 60: 0.9564
Accuracy after query 61: 0.9545
Accuracy after query 62: 0.9535
Accuracy after query 63: 0.9587
Accuracy after query 64: 0.9553
Accuracy after query 65: 0.9552
Accuracy after query 66: 0.9532
Accuracy after query 67: 0.9548
Accuracy after query 68: 0.9602
Accuracy after query 69: 0.9567
Accuracy after query 70: 0.9564
Accuracy after query 71: 0.9589
Accuracy after query 72: 0.9526
Accuracy after query 73: 0.9584
Accuracy after query 74: 0.9577
Accuracy after query 75: 0.9572
Accuracy